# Important References -- also note that much of the code is commented out to reduce runtime:
# `conda activate vgm-data` for this nb (include package file in the future)
* Unable to resolve .show() function. Ignore for now. Several resources were used to resolve this problem. 
    * https://stackoverflow.com/questions/53646669/unable-to-show-scores-in-music21 
    * https://stackoverflow.com/questions/25879764/creating-images-of-notes-in-music21 
    * https://github.com/psychemedia/showntell/blob/music/4.1.0%20Music%20Notation.ipynb 
    * https://github.com/cuthbertLab/music21/issues/260 
* Tutorial Used for Data Prep
    * https://www.datacamp.com/community/tutorials/using-tensorflow-to-compose-music
* Tutorial Use for Parsing (normalOrder attr not used, 'pitches' attr used instead as per www.datacamp.com/ tutorial)
    * https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5
* On the transposition of songs
   * https://stackoverflow.com/questions/37494229/music21-transpose-streams-to-a-given-key

In [1]:
# Imports
from music21 import *
from IPython.display import display
import zipfile
import pickle
import numpy as np

# Functions

In [2]:
def get_n_instrument_measures(instrument_score, instrument_name, n_stop, n_start=0):
    """Gets n measures of an instrument in an instrument-partitioned score.
    
    :param instrument_score: <class 'music21.stream.Score'> a partitioned
        score.
    :param instrument_name: <class 'str'> The name of the desired 
        instrument to extract measures for.
    :param n_stop: <class 'int'> The last measure to get (inclusive).
    :param n_start: <class 'int'> The starting measure to get (0 index).
        DOES NOT SEEM TO BE WORKING. DEFAULTS TO 0 REGARDLESS
    :return: <class 'music21.stream.Score'> 
        The measure-partitioned-instrument-specific score.
    """

    # Validate args
    if (n_start >= n_stop):
        raise ValueError("'n_start' must be less than 'n_stop'")

    # Instrument specific stream
    instrument_stream = instrument_score[instrument_name]

    # Stream containing a range of measures 
    n_measure_stream = instrument_stream.measures(n_start, n_stop)

    # Return the new stream
    return n_measure_stream


def read_and_transpose_midi_to_c(path_to_midi):
    """Transposes a MIDI to C major and returns a score.

    DOES NOT WORK ALL THE TIME.....
    
    :param path_to_midi: <class 'music21.stream.Score'> to
        transposed.
    :return: <class 'music21.stream.Score'> in C major. 
    """
    # Score
    midi_score = converter.parse(path_to_midi)

    # Analyze the key
    midi_key = midi_score.analyze('key')

    # Convert to relative major if minor
    if (str(midi_key).find('minor') != -1):
        midi_key = midi_key.relative

    # Set an interval based on the suspected key's tonic
    transposition_interval = interval.Interval(midi_key.tonic, pitch.Pitch('C'))

    # Check interval and tranpose if not P1 (already key of C major)
    if (transposition_interval.name != 'P1'):
        midi_score = midi_score.transpose(transposition_interval)
    
    # Return the score in the key of C major
    return midi_score

# Config

In [3]:
# Configure
#configure.run()

In [4]:
# Environment settings
env = environment.Environment()

In [5]:
# Set paths
env['musicxmlPath'] = r'C:\Dev\SDK\MuseScore3\bin\MuseScore3.exe'
env['musescoreDirectPNGPath'] = r'C:\Dev\SDK\MuseScore3\bin\MuseScore3.exe'
env['pdfPath'] = r'C:\Dev\SDK\MuseScore3\bin\MuseScore3.exe'
env['graphicsPath'] = r'C:\Dev\SDK\MuseScore3\bin\MuseScore3.exe'
env['lilypondPath'] = r'C:\Dev\SDK\LilyPond\usr\bin\lilypond.exe'

# Initialization and Demonstration of Functions Defined Above -- Transposition is Successful; HOWEVER MuseScore3 Visually Displays the Wrong Key... See Examples and Play The MIDIs for Verification

In [6]:
# Absolute path to a file -- this is a local file and is not on github
abs_path_to_midi_dir = 'C:\Dev\python\CS4850\TEAM2_ORG_LOCAL_DATA\Castlevania'
path_to_all_vg_midis = r'C:\Dev\python\CS4850\TEAM2_ORG_LOCAL_DATA\All_Castlevania'
midi_fnames = os.listdir(abs_path_to_midi_dir)

num = 0
cur_midi = midi_fnames[num]

# Parse the file with music21 
score = converter.parse(os.path.join(abs_path_to_midi_dir, cur_midi))

# Instrument score
instr_score = instrument.partitionByInstrument(score)

# Parts
for part in instr_score.parts:
    print(part)
    
# chordified_score = instr_score.chordify()
# print(type(chordified_score))
# chordified_score.write('midi', './chordified_song.mid')

# # Parse the file and transpose it 
# transposed_score = read_and_transpose_midi_to_c(os.path.join(abs_path_to_midi_dir, cur_midi))
# print('score:', score)
# print('transposed_score:', transposed_score)

<music21.stream.Part Piano>
<music21.stream.Part Electric Bass>
<music21.stream.Part Saxophone>
<music21.stream.Part Timpani>
<music21.stream.Part Tenor Saxophone>
<music21.stream.Part Taiko>
<music21.stream.Part Voice>
<music21.stream.Part StringInstrument>
<music21.stream.Part Oboe>
<music21.stream.Part Church Bells>


In [7]:
# # Analyze keys
# print('Initial Key and Relative Key:', score.analyze('key'), ' -> ', score.analyze('key').relative)
# print('Transposed Key and Relative Key:', transposed_score.analyze('key'), ' -> ', transposed_score.analyze('key').relative)

In [8]:
# # Write the transposed song to file
# transposed_score.write('midi', f'transposed_{cur_midi}')

In [9]:
# # Get certain measures of each score
# # Partition by instrument
# partitioned_transposed_score = instrument.partitionByInstrument(transposed_score)
# n_measure_transposed_score = get_n_instrument_measures(transposed_score, instrument_name='Piano', n_stop=8)
# n_measure_transposed_score.write('midi', f'n_measure_transposed_{cur_midi}')

# Exploration of `music21` Classes and Processing

In [10]:
# # Write the file
# midi.write('midi', f'./{cur_midi}')

In [11]:
# # Check midi
# print(midi)

In [12]:
# def render(s):
#     s.show('lily.png')
#     return Image(filename=str(s.write('lily.png')))

In [13]:
# Make a note
# n1 = note.Note('C4')
#n1.show()

In [14]:
# # Get different instruments
# instruments = instrument.partitionByInstrument(midi)
# print(type(instruments))

In [15]:
# # Print different instruments
# print(type(instruments))
# for instr in instruments:
#     print(instr.partName)

In [16]:
# [part.partName for part in instruments.parts]

In [17]:
# # Extract only certain instrument
# piano_only_stream = instruments['Piano']
# print(piano_only_stream)

In [18]:
# # Write midi with only piano part
# piano_only_stream.write('midi', f'./piano_{cur_midi}')

In [19]:
# # Get only certain measures
# piano_only_8_measure_stream = get_n_instrument_measures(instrument_score=instruments, instrument_name='Piano', n_stop=8)

In [20]:
# # Inspect
# print(piano_only_8_measure_stream)

In [21]:
# # 8 Measure piano to file
# piano_only_8_measure_stream.write('midi', f'8_measure_piano_{cur_midi}')

In [22]:
# # Check key
# piano_only_8_measure_stream.getKeySignatures()

In [23]:
# # Find relative key
# a_minor_key = key.Key('A', 'minor')
# relative_key = a_minor_key.relative
# print(relative_key)
# print(str(a_minor_key).find('minor'))

In [24]:
# # Example -- P1 means perfect unison so they are the same note
# interval.Interval(pitch.Pitch('C'), pitch.Pitch('C'))

In [25]:
# # Transpose a score to C major and make sure to set tonic based on relative major
# transposed_piano_only_8_measure_stream = transpose_score_to_c(piano_only_8_measure_stream)
# print(type(transposed_piano_only_8_measure_stream))

In [26]:
# # Check len key signatures
# transposed_piano_only_8_measure_stream.analyze('key')

In [27]:
# # Write to file and inspect
# transposed_piano_only_8_measure_stream.write('midi', f'transposed_8_measure_{cur_midi}')

In [28]:
# # Part names
# instruments.parts[0].partName

In [29]:
# print(note.Note)
# print(chord.Chord)

In [30]:
# Dictionary of different instruments in the data set and their number of occurences


instrument_dict = {}
for cnt, midi_fname in enumerate(os.listdir(abs_path_to_midi_dir)):

    # Midi file
    midi = converter.parse(os.path.join(abs_path_to_midi_dir, midi_fname))   

    # Streams based on part
    score = instrument.partitionByInstrument(midi)

    # Iterate through parts for a particular song
    for part in score.parts:

        # Validate partName
        if (part.partName):

            # Update dictionary if part not present
            if part.partName not in instrument_dict.keys():
                instrument_dict.update({part.partName: 1})

            # Increment the part if present
            else:
                instrument_dict[part.partName] += 1

In [31]:
# Inspect the dictionary
print("Number of Castlevania MIDIs: ", cnt + 1)
display(instrument_dict)

In [32]:
# Write the data to file
with open('pickled_data/pickled_instruments_dict', 'wb') as fobj:
    pickle.dump(notes, fobj)

# Check what the most common key is for the dataset -- get indices of B- major songs only in music directory

In [63]:
%%time

if (False):
    # Initial variables
    key_signature_count_for_piano_songs_dict = {}
    songs_with_piano = 0

    # Process all files in the aggregated Castlevania music directory
    for midi in os.listdir(path_to_all_vg_midis):

        # Log
        print('Processing:', midi)

        # Parse the file
        score = converter.parse(os.path.join(path_to_all_vg_midis, midi))

        # Parse by instrument
        instrument_score = instrument.partitionByInstrument(score)

        # If a song has a piano part, then increment counter and add key to dictionary
        if ('Piano' in [part.partName for part in instrument_score.parts]):

            # Log
            print('Has Piano')

            # Increment the piano part counter
            songs_with_piano += 1

            # Get the key
            midi_key = score.analyze('key')

            # Convert to relative major if minor
            if (str(midi_key).find('minor') != -1):
                midi_key = midi_key.relative

            ## Update the dictionary
            # Create key if not present
            if (str(midi_key) not in key_signature_count_for_piano_songs_dict.keys()):
                key_signature_count_for_piano_songs_dict.update({str(midi_key): 0})
            
            # Increment value of key if present
            else:
                key_signature_count_for_piano_songs_dict[str(midi_key)] += 1

            # Log
            print(f'For the key of {str(midi_key)} there are', end=' ')
            print(key_signature_count_for_piano_songs_dict[str(midi_key)], ' with that key.')
            
        else:
            # Log
            print('No Piano')

        # Log
        print()

Processing: akumajo1%28V1.1%29.mid
Has Piano
For the key of F major there are 0  with that key.

Processing: akumajo3.mid
Has Piano
For the key of F major there are 1  with that key.

Processing: AkumajouDensetsu_Prayer.mid
Has Piano
For the key of G major there are 0  with that key.

Processing: begining.mid
No Piano

Processing: Bloody_Tears-1.mid
Has Piano
For the key of D- major there are 0  with that key.

Processing: Bloody_Tears.mid
Has Piano
For the key of C major there are 0  with that key.

Processing: c1drac2.mid
No Piano

Processing: c1intro.mid
Has Piano
For the key of G major there are 1  with that key.

Processing: c1vamp.mid
Has Piano
For the key of G major there are 2  with that key.

Processing: c2dracsl.mid
Has Piano
For the key of B major there are 0  with that key.

Processing: c2endng.mid
Has Piano
For the key of E- major there are 0  with that key.

Processing: c2mansio.mid
Has Piano
For the key of B- major there are 0  with that key.

Processing: c2night.mid
Has

In [70]:
# Inspect dictionary and count
# https://stackoverflow.com/questions/268272/getting-key-with-maximum-value-in-dictionary
if (False):
    print('Number of songs:', len(os.listdir(path_to_all_vg_midis)))
    print('Number of songs with piano:', songs_with_piano)
    display(key_signature_count_for_piano_songs_dict)
    print('The key with the greatest number of songs is:', 
        max(key_signature_count_for_piano_songs_dict, key=key_signature_count_for_piano_songs_dict.get))
    print('Total in dictionary:', sum(key_signature_count_for_piano_songs_dict.values()))

Number of songs: 177
Number of songs with piano: 83


{'F major': 12,
 'G major': 11,
 'D- major': 2,
 'C major': 7,
 'B major': 1,
 'E- major': 8,
 'B- major': 19,
 'A major': 3,
 'G- major': 0,
 'E major': 2,
 'A- major': 3,
 'D major': 3}

The key with the greatest number of songs is: B- major
Total in dictionary: 71


In [71]:
if (False):
    # Make summary dictionary for this analysis and pickle it
    data_summary_dict = {
        'num_songs': len(os.listdir(path_to_all_vg_midis)),
        'num_songs_with_piano': songs_with_piano,
        'key_signature_dict': key_signature_count_for_piano_songs_dict
    }
    with open('./pickled_data/pickled_data_summary_dict', 'wb') as fobj:
        pickle.dump(data_summary_dict, fobj)

In [79]:
## Load the pickled data summary 
# The most commmon key is B- major
with open('./pickled_data/pickled_data_summary_dict', 'rb') as fobj:
    load_data_summary_dict = pickle.load(fobj)
display(load_data_summary_dict)

{'num_songs': 177,
 'num_songs_with_piano': 83,
 'key_signature_dict': {'F major': 12,
  'G major': 11,
  'D- major': 2,
  'C major': 7,
  'B major': 1,
  'E- major': 8,
  'B- major': 19,
  'A major': 3,
  'G- major': 0,
  'E major': 2,
  'A- major': 3,
  'D major': 3}}

In [83]:
# Get indices of only dominant key
dominant_key = 'B- major'
indices_of_dominant_key = []

# Process all files in the aggregated Castlevania music directory
for ix, midi in enumerate(os.listdir(path_to_all_vg_midis)):

    # Log
    print('Processing:', midi)

    # Parse the file
    score = converter.parse(os.path.join(path_to_all_vg_midis, midi))

    # Parse by instrument
    instrument_score = instrument.partitionByInstrument(score)

    # Check for piano
    if ('Piano' in [part.partName for part in instrument_score.parts]):
        
        # Log
        print('Has Piano')

        # Check key signature
        key_sig = instrument_score.analyze('key')

        # Convert to relative major if minor
        if (str(key_sig).find('minor') != -1):
            key_sig = key_sig.relative

        # Log
        print('This song is in the key of', key_sig)

        # Check to see if the index should be recorded
        if (str(key_sig) == dominant_key):

            # Log
            print('Adding to indices list')

            # Add the index of this song in the music directory to a list
            indices_of_dominant_key.append(ix)

    else:

        # Log
        print('Does NOT have Piano')
    
    # Log
    print()

Processing: akumajo1%28V1.1%29.mid
Has Piano
This song is in the key of F major

Processing: akumajo3.mid
Has Piano
This song is in the key of F major

Processing: AkumajouDensetsu_Prayer.mid
Has Piano
This song is in the key of G major

Processing: begining.mid
Does NOT have Piano

Processing: Bloody_Tears-1.mid
Has Piano
This song is in the key of D- major

Processing: Bloody_Tears.mid
Has Piano
This song is in the key of C major

Processing: c1drac2.mid
Does NOT have Piano

Processing: c1intro.mid
Has Piano
This song is in the key of G major

Processing: c1vamp.mid
Has Piano
This song is in the key of G major

Processing: c2dracsl.mid
Has Piano
This song is in the key of B major

Processing: c2endng.mid
Has Piano
This song is in the key of E- major

Processing: c2mansio.mid
Has Piano
This song is in the key of B- major
Adding to indices list

Processing: c2night.mid
Has Piano
This song is in the key of G major

Processing: c2pword.mid
Does NOT have Piano

Processing: c2town.mid
Has 

In [84]:
# Check indices
display(indices_of_dominant_key)
print(len(indices_of_dominant_key))

[11,
 14,
 18,
 27,
 58,
 61,
 62,
 64,
 71,
 72,
 79,
 85,
 92,
 130,
 144,
 147,
 152,
 157,
 174]

19


In [94]:
# Add to the dictionary the list of indices with the dominant key and write that data
if (False):
    load_data_summary_dict.update({'indices_of_predominant_key': indices_of_dominant_key})
    with open('./pickled_data/pickled_data_summary_dict', 'wb') as fobj:
        pickle.dump(load_data_summary_dict, fobj)

In [95]:
# Load the new data just to check
with open('./pickled_data/pickled_data_summary_dict', 'rb') as fobj:
    load_data_summary_dict = pickle.load(fobj)
load_data_summary_dict

{'num_songs': 177,
 'num_songs_with_piano': 83,
 'key_signature_dict': {'F major': 12,
  'G major': 11,
  'D- major': 2,
  'C major': 7,
  'B major': 1,
  'E- major': 8,
  'B- major': 19,
  'A major': 3,
  'G- major': 0,
  'E major': 2,
  'A- major': 3,
  'D major': 3},
 'indices_of_predominant_key': [11,
  14,
  18,
  27,
  58,
  61,
  62,
  64,
  71,
  72,
  79,
  85,
  92,
  130,
  144,
  147,
  152,
  157,
  174]}

# Summary of Data Collected So Far

In [ ]:
## Data summary dict
with open('./pickled_data/pickled_data_summary_dict', 'rb') as fobj:
    load_data_summary_dict = pickle.load(fobj)
display(load_data_summary_dict)

#
piano_scores_list = []
indices_of_dominant_key = 
for midi_fname in [os.listdir(path_to_all_vg_midis)[i] for i in indices_of_dominant_key]:

# Thoughts About Above Data
B- major is the dominant key and only 19 songs have that key. Consider expanding the dataset, but for now just build based on those chords. Transposition might be needed later, but for now isn't worth figuring out.

# Get N Measures of a Specific Instrument, Parse by Notes/Chords, then Pickle It

In [102]:
%%time
## Get transposed, n-measure, piano only scores for the Castlevania Dataset
## then append to a list of containing all those scores. 

# Flags for this procedure
analysis_flag = True
write_flag = False

# Score converted to strings representing notes and chords
dominant_key = 'bflatmaj'
indices_of_dominant_key = load_data_summary_dict['indices_of_predominant_key']
n_measure_piano_scores_list = []
n = 8
instr_name = 'Piano'
if (analysis_flag):

    # Iterate through files and build array of notes
    for midi_fname in [os.listdir(path_to_all_vg_midis)[i] for i in indices_of_dominant_key]:
        # Processing a midi file
        print('Processing: ', midi_fname)

        # The midi file itself converted and parsed into a score
        score = converter.parse(os.path.join(path_to_all_vg_midis, midi_fname))

        # Streams based on Part (Piano, Voice, StringedInstrument, etc.)
        instrument_score = instrument.partitionByInstrument(score)

        ## No need to check for piano since the files provided MUST have piano based on preprocessing

        # Get n measures of an instrument stream 
        piano_eight_measure_score = get_n_instrument_measures(instrument_score=instrument_score, instrument_name=instr_name, n_stop=n)

        # Chordify those n measures of the instrument score... unsure of efficacy of this...
        piano_eight_measure_score = piano_eight_measure_score.chordify()

        # Append to the scores list
        n_measure_piano_scores_list.append(piano_eight_measure_score)

        # Whitespace for clarity
        print()

Processing:  c2mansio.mid

Processing:  c2town.mid

Processing:  c3cont.mid

Processing:  c3everg.mid

Processing:  Castlevania_Out_of_Time.mid

Processing:  Castlevania_Vampire_Killer.mid

Processing:  Castlevania_Vampire_Killer_Arranged.mid

Processing:  casv2-3.mid

Processing:  cv1-1b.mid

Processing:  cv1-1d.mid

Processing:  cv1-6.mid

Processing:  cv1intro.mid

Processing:  cv2-town.mid

Processing:  cv_timeout.mid

Processing:  j47_castlevania_orchesteral.mid

Processing:  JILost_-_NES_-_Castlevania_-_Stalker_remix.mid

Processing:  SilenceofDaylight.mid

Processing:  SQ_thesilenceofdaylight.mid

Processing:  WickedChild.mid

Wall time: 22.5 s


In [34]:
# # Inspect list
# # Processing of vampire killer took a particularly long time
# print(len(transposed_n_measure_piano_scores_list))

In [103]:
# Pickle the result the n measure scores
if (True):
    with open(f'./pickled_data/pickled_{dominant_key}_{n}_measure_piano_scores_list', 'wb') as fobj:
        pickle.dump(n_measure_piano_scores_list, fobj)

# Pick Out Chords, Notes, and Durations

In [104]:
# Load the list to check length
with open(f'./pickled_data/pickled_{dominant_key}_{n}_measure_piano_scores_list', 'rb') as fobj:
    scores_list = pickle.load(fobj)

In [105]:
# Check the keys of these pieces -- two songs with nothing written in them...
if (False):
    write_path = r'C:\Dev\python\CS4850\TEAM2_ORG_LOCAL_DATA\Transposed_Songs'
    for ix, score in enumerate(scores_list):
        score.write('midi', os.path.join(f'{write_path}', f'{ix}.mid'))

In [116]:
## Note: Both these data structures are normalized in terms of key
## so they can be used for training 

# Empty lists of lists for chords and durations
# Each element of the list is a list of chords (or durations) respectively for 
# a given score
chords_ds = [[] for _ in scores_list]   # Holds chords and notes
durations_ds = [[] for _ in scores_list]

In [117]:
# Inspect data structures
print(chords_ds)
print(durations_ds)

[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]


In [118]:
# Extract notes and chords
for ix, score in enumerate(scores_list):

    # All notes/chords in the score
    notes_to_parse = score.recurse()

    for element in notes_to_parse:

        if (isinstance(element, note.Note)):

            # Append to lists
            chords_ds[ix].append(element.pitch)
            durations_ds[ix].append(element.duration.quarterLength)
        elif (isinstance(element, chord.Chord)):

            # Append to lists
            chords_ds[ix].append(''.join(str(n) for n in element.pitches))  # .normalOrder attr would work here, too
            durations_ds[ix].append(element.duration.quarterLength)
    
    # Track progress
    print('The n=', ix, ' extraction of notes, chords, and durations is complete.')

The n= 0  extraction of notes, chords, and durations is complete.
The n= 1  extraction of notes, chords, and durations is complete.
The n= 2  extraction of notes, chords, and durations is complete.
The n= 3  extraction of notes, chords, and durations is complete.
The n= 4  extraction of notes, chords, and durations is complete.
The n= 5  extraction of notes, chords, and durations is complete.
The n= 6  extraction of notes, chords, and durations is complete.
The n= 7  extraction of notes, chords, and durations is complete.
The n= 8  extraction of notes, chords, and durations is complete.
The n= 9  extraction of notes, chords, and durations is complete.
The n= 10  extraction of notes, chords, and durations is complete.
The n= 11  extraction of notes, chords, and durations is complete.
The n= 12  extraction of notes, chords, and durations is complete.
The n= 13  extraction of notes, chords, and durations is complete.
The n= 14  extraction of notes, chords, and durations is complete.
The n

# Encode the Data with One-Hot Encoding.

In [46]:
# Load data and remove songs with low note counts
with open('./pickled_data/pickled_feature_and_encoding_dict', 'rb') as fobj:
    reduced_features_dict = pickle.load(fobj)

In [47]:
# Load chord and duration_ds
chords_ds = reduced_features_dict['chords_ds']
durations_ds = reduced_features_dict['durations_ds']

In [48]:
## Get the length of each/chord note sublist
# ... NO NEED TO DO THIS SINCE THE LISTS ARE PARALLEL
indices_of_removed_songs_list = [] 
print('Before sorting', [len(sublist) for sublist in chords_ds])
song_lengths_in_chords_list = sorted([len(sublist) for sublist in chords_ds])
print('After Sorting but Before Deletion:', song_lengths_in_chords_list)
for ix, song_length in enumerate(song_lengths_in_chords_list):
    if (song_length < 16):
        indices_of_removed_songs_list.append(ix)

print('After deletion', song_lengths_in_chords_list[max(indices_of_removed_songs_list)+1:])
print('Total chords/notes:', sum(song_lengths_in_chords_list))
print('Indices of removed songs:', indices_of_removed_songs_list)

Before sorting [68, 103, 5, 40, 32, 92, 121, 93, 80, 123, 0, 42, 93, 32, 74, 108, 50, 98, 0]
After Sorting but Before Deletion: [0, 0, 5, 32, 32, 40, 42, 50, 68, 74, 80, 92, 93, 93, 98, 103, 108, 121, 123]
After deletion [32, 32, 40, 42, 50, 68, 74, 80, 92, 93, 93, 98, 103, 108, 121, 123]
Total chords/notes: 1254
Indices of removed songs: [0, 1, 2]


In [49]:
# Remove the desired songs in each datastructure and get only songs longer than a given
# sequence of notes. Therefore, REDUCE the number of songs to worry about
if ([len(ele) for ele in reduced_chords_ds] == [len(ele) for ele in reduced_durations_ds]):
    print(True)
reduced_chords_ds = [sublist for sublist in chords_ds if len(sublist) > 16]
reduced_durations_ds = [sublist for sublist in durations_ds if len(sublist) > 16]

True


In [50]:
### Encoding map

## Mapping chords to integers
# The chords_ds is a list of lists so to find unique elements (i.e. chord/note combos)
# in the data, I must inspect an 
unique_chords = np.unique([item for sublist in reduced_chords_ds for item in sublist])  # Flatten list of lists
chord_to_int = dict(zip(unique_chords, list(range(0, len(unique_chords)))))  # Uncertain

## Map unique durations to integers
unique_durations = np.unique([item for sublist in reduced_durations_ds for item in sublist])
duration_to_int = dict(zip(unique_durations, list(range(0, len(unique_durations)))))

# Print number of unique notes and chords
print(len(unique_chords))

# Print number of unique durations
print(len(unique_durations))

202
12


In [51]:
# Inspect the items (key:value pairs) in the dictionary
list(chord_to_int.items())[:10]

[('A2B-2F3B-3D4F4C5G5', 0),
 ('A2B-2F3D4F4C5G5', 1),
 ('A3B-4', 2),
 ('A3C4', 3),
 ('A3C5', 4),
 ('A3E4', 5),
 ('A3F#4', 6),
 ('A3F4', 7),
 ('A3G4', 8),
 ('A4', 9)]

In [52]:
### Decoding map
int_to_chord = {integer: chord for chord, integer in chord_to_int.items()}
int_to_duration = {integer: duration for duration, integer in duration_to_int.items()}

In [53]:
# Pickle all this together
feature_and_encoding_dict = {
    'reduced_chords_ds': reduced_chords_ds,
    'reduced_durations_ds': reduced_durations_ds,
    'chord_to_int': chord_to_int,
    'duration_to_int': duration_to_int,
    'int_to_chord': int_to_chord,
    'int_to_duration': int_to_duration,
}

In [54]:
# Pickle it all up
with open('./pickled_data/pickled_feature_and_encoding_dict', 'wb') as fobj:
    pickle.dump(feature_and_encoding_dict, fobj)